In [ ]:
import pandas as pd, numpy as np
from pathlib import Path

BASE = Path(r"C:\Users\maria\Documents\VANGUARD DATA")

# dataset with no inconsistencies (KPIs)
events_fp = BASE / "analysis_ready_events.csv"
if events_fp.exists():
    w = pd.read_csv(events_fp, parse_dates=["date_time"])
else:

    w   = pd.read_csv(BASE / "df_final_web_data_clean.csv", parse_dates=["date_time"])
    exp = pd.read_csv(BASE / "df_final_experiment_clients_clean.csv")
    # mapping
    v = exp["variation"].astype(str).str.lower().str.strip()
    mapping = {"control":"Control","variant a":"Control","a":"Control","0":"Control",
               "test":"Test","treatment":"Test","variant b":"Test","b":"Test","1":"Test"}
    exp["group"] = v.map(mapping).fillna("Unknown")
    w = w.merge(exp[["client_id","group"]], on="client_id", how="left")

# Keep just Control/Test for A/B
w = w[w["group"].isin(["Control","Test"])].copy()

# Build a client-level table
per_client = (w.groupby(["group","client_id"])
                .agg(completed=("process_step", lambda s: (s=="confirm").any()))
                .reset_index())
per_client.head()

,group,client_id,completed
0,Control,1028,False
1,Control,1104,False
2,Control,1186,False
3,Control,1195,True
4,Control,1197,True


In [ ]:
tab = per_client.groupby("group")["completed"].agg(["sum","count","mean"]).rename(columns={"mean":"rate"})
print(tab)
p_control = tab.loc["Control","rate"]; n_control = int(tab.loc["Control","count"]); x_control = int(tab.loc["Control","sum"])
p_test    = tab.loc["Test","rate"];    n_test    = int(tab.loc["Test","count"]);    x_test    = int(tab.loc["Test","sum"])

print("\nClient completion rates:")
print(f"Control: {p_control:.4f}  (x={x_control}, n={n_control})")
print(f"Test:    {p_test:.4f}     (x={x_test}, n={n_test})")
print(f"Diff (Test - Control): {(p_test - p_control):.4f}")

           sum  count      rate
group                          
Control  15226  23406  0.650517
Test     18427  26867  0.685860

Client completion rates:
Control: 0.6505  (x=15226, n=23406)
Test:    0.6859     (x=18427, n=26867)
Diff (Test - Control): 0.0353


In [ ]:
tab = per_client.groupby("group")["completed"].agg(["sum","count","mean"]).rename(columns={"mean":"rate"})
print(tab)
p_control = tab.loc["Control","rate"]; n_control = int(tab.loc["Control","count"]); x_control = int(tab.loc["Control","sum"])
p_test    = tab.loc["Test","rate"];    n_test    = int(tab.loc["Test","count"]);    x_test    = int(tab.loc["Test","sum"])

print("\nClient completion rates:")
print(f"Control: {p_control:.4f}  (x={x_control}, n={n_control})")
print(f"Test:    {p_test:.4f}     (x={x_test}, n={n_test})")
print(f"Diff (Test - Control): {(p_test - p_control):.4f}")

           sum  count      rate
group                          
Control  15226  23406  0.650517
Test     18427  26867  0.685860

Client completion rates:
Control: 0.6505  (x=15226, n=23406)
Test:    0.6859     (x=18427, n=26867)
Diff (Test - Control): 0.0353


In [ ]:
from math import sqrt
from statistics import fmean
import math

# Inputs (from cell before)
# x_test, n_test, x_control, n_control

p_pool = (x_test + x_control) / (n_test + n_control)
se_pooled = math.sqrt(p_pool * (1 - p_pool) * (1/n_test + 1/n_control))

z = (p_test - p_control) / se_pooled
# One-sided p-value (Test > Control)
from math import erf, sqrt
def norm_cdf(z):  # standard normal CDF
    return 0.5 * (1 + erf(z / sqrt(2)))

p_value = 1 - norm_cdf(z)

# 95% CI
se_unpooled = math.sqrt(p_test*(1-p_test)/n_test + p_control*(1-p_control)/n_control)
ci_low  = (p_test - p_control) - 1.96*se_unpooled
ci_high = (p_test - p_control) + 1.96*se_unpooled

print(f"z = {z:.3f}")
print(f"one-sided p-value (Test > Control) = {p_value:.6f}")
print(f"95% CI for (Test - Control): [{ci_low:.4f}, {ci_high:.4f}]")
alpha = 0.05
print("Decision:", "Reject H0 (Test > Control)" if p_value < alpha else "Fail to reject H0")

z = 8.403
one-sided p-value (Test > Control) = 0.000000
95% CI for (Test - Control): [0.0271, 0.0436]
Decision: Reject H0 (Test > Control)


In [ ]:
delta0 = 0.05
diff = p_test - p_control
z_thr = (diff - delta0) / se_unpooled
p_thr = 1 - norm_cdf(z_thr)

print(f"Observed diff: {diff:.4f}")
print(f"Test against 0.0500 → z = {z_thr:.3f}, p(one-sided) = {p_thr:.6f}")
print("Decision (α=0.05):", "Meets/exceeds +5pp (reject H0)" if p_thr < 0.05 else "Does NOT meet +5pp (fail to reject H0)")

Observed diff: 0.0353
Test against 0.0500 → z = -3.481, p(one-sided) = 0.999750
Decision (α=0.05): Does NOT meet +5pp (fail to reject H0)


In [ ]:
import pandas as pd
from scipy.stats import ttest_ind

# 1) normalize types
demo = pd.read_csv(BASE / "df_final_demo_clean.csv")
demo["client_id"] = demo["client_id"].astype(str).str.strip()
demo["client_id"] = demo["client_id"].str.replace(r"\.0$", "", regex=True)  # in case some came from floats
demo["clnt_age"]  = pd.to_numeric(demo["clnt_age"], errors="coerce")

# 2) Normalize per_client client_id
per_client["client_id"] = per_client["client_id"].astype(str).str.strip()
per_client["client_id"] = per_client["client_id"].str.replace(r"\.0$", "", regex=True)

demo_age = (demo[["client_id","clnt_age"]]
            .drop_duplicates(subset=["client_id"], keep="first"))

age_df = per_client.merge(demo_age, on="client_id", how="left", validate="m:1")

print("Dtypes:", per_client["client_id"].dtype, demo_age["client_id"].dtype)
print("Rows merged:", len(age_df),
      "| missing ages:", age_df["clnt_age"].isna().sum())

age_c = age_df.loc[age_df["group"]=="Control", "clnt_age"].dropna()
age_t = age_df.loc[age_df["group"]=="Test",    "clnt_age"].dropna()

print(f"Control n={len(age_c)}, mean={age_c.mean():.2f}, std={age_c.std():.2f}")
print(f"Test    n={len(age_t)}, mean={age_t.mean():.2f}, std={age_t.std():.2f}")

if len(age_c) > 1 and len(age_t) > 1:
    tstat, pval = ttest_ind(age_t, age_c, equal_var=False, nan_policy="omit")  # Welch
    print(f"Welch t-test: t={tstat:.3f}, two-sided p={pval:.6f}")
    print("Decision (α=0.05):", "Different means (reject H0)" if pval<0.05 else "No evidence of difference (fail to reject H0)")
else:
    print("Not enough non-missing age data to run the test.")

Dtypes: object object
Rows merged: 50273 | missing ages: 13
Control n=23400, mean=47.51, std=15.52
Test    n=26860, mean=47.18, std=15.51
Welch t-test: t=-2.367, two-sided p=0.017913
Decision (α=0.05): Different means (reject H0)


In [ ]:
# Group sizes
print("Clients per group:", per_client["group"].value_counts().to_dict())

demo["clnt_tenure_yr"] = pd.to_numeric(demo["clnt_tenure_yr"], errors="coerce")
bal = per_client.merge(demo[["client_id","clnt_age","clnt_tenure_yr","gendr"]], on="client_id", how="left")

def quick_desc(s):
    return pd.Series({"n":s.notna().sum(), "mean":s.mean(), "median":s.median()})

print("\nAge by group:\n", bal.groupby("group")["clnt_age"].apply(quick_desc))
print("\nTenure by group:\n", bal.groupby("group")["clnt_tenure_yr"].apply(quick_desc))
print("\nGender share by group:\n", bal.groupby("group")["gendr"].value_counts(normalize=True).round(3).unstack(fill_value=0))

Clients per group: {'Test': 26867, 'Control': 23406}

Age by group:
 group          
Control  n         23400.000000
         mean         47.506795
         median       48.500000
Test     n         26860.000000
         mean         47.178276
         median       47.500000
Name: clnt_age, dtype: float64

Tenure by group:
 group          
Control  n         23401.000000
         mean         12.088244
         median       11.000000
Test     n         26860.000000
         mean         11.984847
         median       11.000000
Name: clnt_tenure_yr, dtype: float64

Gender share by group:
 gendr        F      M      U    X
group                            
Control  0.320  0.339  0.341  0.0
Test     0.323  0.333  0.344  0.0


In [ ]:
# Daily visits and conversion
v = (w.groupby("visit_id")
       .agg(group=("group","first"),
            completed=("process_step", lambda s: (s=="confirm").any()),
            dt=("date_time","min"))
       .reset_index())
daily = v.groupby(v["dt"].dt.date).agg(visits=("visit_id","count"), conv=("completed","mean"))
print(daily.describe().round(3))

# 95% CI half-width for overall completion
p = per_client["completed"].mean()
n = per_client.shape[0]
import math
half_width = 1.96 * math.sqrt(p*(1-p)/n)
print(f"Overall completion ~ {p:.3f} ± {half_width:.3f} (95% CI)")

         visits    conv
count    97.000  97.000
mean    709.495   0.474
std     984.044   0.056
min      45.000   0.317
25%     280.000   0.442
50%     363.000   0.470
75%     651.000   0.500
max    6408.000   0.623
Overall completion ~ 0.669 ± 0.004 (95% CI)


In [ ]:
# counts per_client table
tab = per_client.groupby("group")["completed"].agg(["sum","count"])
x_c, n_c = int(tab.loc["Control","sum"]), int(tab.loc["Control","count"])
x_t, n_t = int(tab.loc["Test","sum"]),    int(tab.loc["Test","count"])

from statsmodels.stats.proportion import proportions_ztest, confint_proportions_2indep

# 1) Is Test > Control?
z, p = proportions_ztest([x_t, x_c], [n_t, n_c], alternative="larger")
ci = confint_proportions_2indep(x_t, n_t, x_c, n_c, method="wald")
print(f"Diff={x_t/n_t - x_c/n_c:.4f}, z={z:.3f}, p(one-sided)={p:.6f}, CI95%={ci}")

# 2) Meets +5pp?
z5, p5 = proportions_ztest([x_t, x_c], [n_t, n_c], value=0.05, alternative="larger")
print(f"vs +5pp: z={z5:.3f}, p(one-sided)={p5:.6f}")

Diff=0.0353, z=8.403, p(one-sided)=0.000000, CI95%=(np.float64(0.027089623709934507), np.float64(0.043596407211046406))
vs +5pp: z=-3.485, p(one-sided)=0.999754


In [ ]:
from scipy.stats import chisquare
obs = per_client["group"].value_counts().loc[["Control","Test"]].values
exp = obs.sum() * np.array([0.5, 0.5])  # expected 50/50
chi2, p = chisquare(obs, f_exp=exp)
print("SRM:", dict(zip(["Control","Test"], obs)), "| chi2=", round(chi2,2), "p=", p)

SRM: {'Control': np.int64(23406), 'Test': np.int64(26867)} | chi2= 238.27 p= 9.376573639032581e-54


In [ ]:
v = (w.groupby(["client_id","group","visit_id"])
       .agg(completed=("process_step", lambda s: (s=="confirm").any()),
            dt=("date_time","min")).reset_index())
v = v.sort_values("dt")
v["visit_num"] = v.groupby("client_id")["dt"].rank(method="first").astype(int)

first = v[v["visit_num"]==1]
tab = first.groupby("group")["completed"].agg(["sum","count"])
from statsmodels.stats.proportion import proportions_ztest
z, p = proportions_ztest([tab.loc["Test","sum"], tab.loc["Control","sum"]],
                         [tab.loc["Test","count"], tab.loc["Control","count"]],
                         alternative="larger")
print("First-visit completion:", tab.assign(rate=tab["sum"]/tab["count"]), "| z=", round(z,3), "p=", p)

First-visit completion:            sum  count      rate
group                          
Control  12469  23406  0.532727
Test     15226  26867  0.566718 | z= 7.643 p= 1.0617052973488282e-14


In [ ]:
ws = w.sort_values(["visit_id","date_time"])
prev = ws.groupby("visit_id")["step_order"].shift()
back = (ws["step_order"] < prev)
visit_back = ws.assign(back=back).groupby(["group","visit_id"])["back"].any().reset_index()
tab = visit_back.groupby("group")["back"].agg(["sum","count"])
z, p = proportions_ztest([tab.loc["Test","sum"], tab.loc["Control","sum"]],
                         [tab.loc["Test","count"], tab.loc["Control","count"]],
                         alternative="smaller")
print("Backtracking:", tab.assign(rate=tab["sum"]/tab["count"]), "| z=", round(z,3), "p=", p)

Backtracking:           sum  count      rate
group                         
Control  6531  31967  0.204304
Test     9976  36974  0.269811 | z= 20.099 p= 1.0


In [ ]:
reached3 = (w[w["process_step"]=="step_3"][["group","visit_id"]].drop_duplicates())
confirmed = (w[w["process_step"]=="confirm"][["group","visit_id"]].drop_duplicates())
x = reached3.merge(confirmed, on=["group","visit_id"], how="left", indicator=True)
x["y"] = (x["_merge"]=="both")
tab = x.groupby("group")["y"].agg(["sum","count"])
z, p = proportions_ztest([tab.loc["Test","sum"], tab.loc["Control","sum"]],
                         [tab.loc["Test","count"], tab.loc["Control","count"]],
                         alternative="larger")
print("Step3→Confirm:", tab.assign(rate=tab["sum"]/tab["count"]), "| z=", round(z,3), "p=", p)

Step3→Confirm:            sum  count      rate
group                          
Control  15027  18031  0.833398
Test     18078  21880  0.826234 | z= -1.894 p= 0.9708772794354223


In [ ]:
from scipy.stats import mannwhitneyu
import numpy as np

# build visit-level TTC in minutes
t1 = w.groupby(["group","visit_id"])["date_time"].agg(["min","max"]).dropna()
t1["ttc_min"] = (t1["max"] - t1["min"]).dt.total_seconds()/60

# pick Control/Test series (drop NaN just in case)
c = t1.loc["Control", "ttc_min"].dropna()
t = t1.loc["Test",    "ttc_min"].dropna()

# Mann–Whitney
u, p = mannwhitneyu(t, c, alternative="less")

print(f"TTC median (min): Control={c.median():.2f}  Test={t.median():.2f}  |  U={u:.0f}  p={p:.6f}")

TTC median (min): Control=2.67  Test=2.80  |  U=616442292  p=1.000000


In [ ]:
# visits
vis = (w.groupby(["client_id","group","visit_id"])
         .agg(start=("date_time","min"),
              completed=("process_step", lambda s: (s=="confirm").any()))
         .reset_index())
vis["date"] = vis["start"].dt.date
vis.to_csv(BASE / "tableau_visits.csv", index=False)

# daily
daily = vis.groupby("date").agg(visits=("visit_id","count"), completion=("completed","mean")).reset_index()
daily.to_csv(BASE / "tableau_daily.csv", index=False)
print("Saved tableau_visits.csv & tableau_daily.csv")

Saved tableau_visits.csv & tableau_daily.csv
